### Attention

在之前的Seq2Seq中简单介绍了注意力机制，其通过在每个时间步中为输入序列中不同位置袋词分配不同的注意力权重。使得模型能够更加灵活地有选择地关注输入序列中的重要部分，从而更好地捕捉上下文相关性，模型的性能也会因此而提高。  

在Attention中最重要的就是如何计算得分，下图中列出了三种计算score的方法:
<img src="./images/score_function.png" alt="examples" style="zoom:30%;" />  
其中最简单的就是点积注意力，所以以点积注意力为例介绍下计算两个张量的点积注意力的步骤

#### Dot Product Attention

点积注意力顾名思义就是计算两个向量的点积

> 1. 创建两个张量: x1和x2

In [1]:
import torch
x1 = torch.randn(2, 3, 4) # (batch_size, seq_len1,  feature_dim)
x2 = torch.randn(2, 5, 4) # (batch_size, seq_len2,  feature_dim)

print(x1)
print(x2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[[-0.5337, -0.4676, -0.2803, -0.6368],
         [ 0.4674, -0.6213,  1.5579,  0.2032],
         [-0.1544, -0.9061,  0.7596, -0.0978]],

        [[ 1.7761,  1.1352,  0.7950,  1.2606],
         [ 0.2405,  1.2241, -0.0210, -1.2548],
         [-0.2855, -1.1407, -1.2737, -0.9245]]])
tensor([[[-1.9028, -1.1113,  0.9790,  1.7046],
         [ 0.0688,  1.5732,  0.0434, -0.3994],
         [-1.2683,  0.6892,  0.7112,  1.4494],
         [-0.1494,  1.1914, -0.1581,  0.4710],
         [-1.2611, -0.0855, -1.4043, -0.2682]],

        [[ 0.1631, -0.7872,  0.7288,  1.4120],
         [ 0.1255, -0.1833,  0.8956, -1.5014],
         [-0.6591, -0.4399,  0.4521,  0.7559],
         [ 0.9043, -0.0518, -0.5727,  0.3202],
         [-0.3043,  0.3897, -0.0329, -0.7944]]])


> 2. 计算点积, 得到原始权重(score)  

weight中包含x1每一行和x2中每一列的点积结果, 表征了任意两组向量的其相似程度, 权重矩阵的每个元素值也就是相似度得分

<img src="./images/dot_product.png" alt="examples" style="zoom:60%;" />


In [4]:
raw_weights = torch.bmm(x1, x2.transpose(1,2))
print(raw_weights) # (2, 3, 5)

tensor([[[ 0.1752, -0.5302, -0.7677, -0.7330,  1.2773],
         [ 1.6725, -0.9587,  0.3814, -0.9606, -2.7785],
         [ 1.8777, -1.3641, -0.0301, -1.2226, -0.7683]],

        [[ 1.7554, -1.1658, -0.3576,  1.4957, -1.1257],
         [-2.7115,  1.6709, -1.6550, -0.2356,  1.4014],
         [-1.3823,  0.4206, -0.5847,  0.2342,  0.4187]]])


> 3. 对原始权重进行归一化

对权重矩阵进行归一化处理, 得到概率分布统计

In [5]:
import torch.nn.functional as F

attn_weights = F.softmax(raw_weights, dim=-1)
print(attn_weights)

tensor([[[0.1888, 0.0932, 0.0735, 0.0761, 0.5683],
         [0.6991, 0.0503, 0.1922, 0.0502, 0.0082],
         [0.7672, 0.0300, 0.1139, 0.0346, 0.0544]],

        [[0.4995, 0.0269, 0.0604, 0.3853, 0.0280],
         [0.0064, 0.5100, 0.0183, 0.0758, 0.3895],
         [0.0491, 0.2977, 0.1089, 0.2471, 0.2972]]])


> 4. 求注意力分布的加权和  

将权重矩阵与x2相乘就得到了加权和，此时的输出与x1的形状相同，可以看作是关注了x2之后新x1  

<img src="./images/attention_output.png" alt="examples" style="zoom:60%;" />

In [6]:
attn_output = torch.bmm(attn_weights, x2)
print(attn_output)

tensor([[[-1.1741,  0.0297, -0.5690,  0.2746],
         [-1.5883, -0.5060,  0.8039,  1.4716],
         [-1.6759, -0.6904,  0.7514,  1.4625]],

        [[ 0.3849, -0.4337,  0.1938,  0.8116],
         [ 0.0029,  0.0413,  0.4135, -1.0280],
         [ 0.1066, -0.0381,  0.2004, -0.4523]]])


#### Scaled Dot Production Attention
缩放点积注意力与点积注意力的主要区别是：缩放点积注意力在于计算注意力权重之前，会将点积结果也就是原始权重(得分)除以一个缩放因子，得到缩放后的原始权重，通常这个缩放因子是输入特征维度的平方根。  
之所以使用缩放因子，是因为在深度学习模型中，点积的值可能非常大，这样会引起梯度消失的情况。因此通过缩放因子可以确保softmax函数在一个较为平缓的区域内工作，减轻梯度消失的问题

In [7]:
import torch # 导入 torch
import torch.nn.functional as F # 导入 nn.functional
# 1. 创建两个张量 x1 和 x2
x1 = torch.randn(2, 3, 4) # 形状 (batch_size, seq_len1, feature_dim)
x2 = torch.randn(2, 5, 4) # 形状 (batch_size, seq_len2, feature_dim)
# 2. 计算张量点积，得到原始权重
raw_weights = torch.bmm(x1, x2.transpose(1, 2)) # 形状 (batch_size, seq_len1, seq_len2)
# 3. 将原始权重除以缩放因子
scaling_fator = x1.size(-1) ** 0.5
scaled_weights = raw_weights / scaling_fator
# 4. 归一化
attn_weights = F.softmax(raw_weights, dim=-1)
# 5. 使用注意力权重对x2加权求和
attn_output = torch.bmm(attn_weights, x2)
print(attn_output)

tensor([[[ 0.2498,  0.4867, -0.0678, -0.3209],
         [ 0.3602, -0.8344,  2.1514,  0.5412],
         [-0.0681, -0.6443,  1.7880,  0.1678]],

        [[-0.7950, -0.0455, -0.0373, -0.6455],
         [-0.3184,  0.2039, -0.2105, -0.6514],
         [ 0.2170, -1.7193, -1.1416, -1.7688]]])
